In [1]:
import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import HuberRegressor
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv('final_df_binary.csv')

In [3]:
print(df.head())

         date section_id  employees_on_duty  total_task_time_minutes  \
0  2021-01-01    SEC-001                  1               834.569483   
1  2021-01-01    SEC-002                  4              1525.368990   
2  2021-01-01    SEC-003                  2               662.095279   
3  2021-01-01    SEC-004                  4              1499.112620   
4  2021-01-01    SEC-005                  3              1238.533073   

   is_holiday  is_weekend  
0           0           0  
1           0           0  
2           0           0  
3           0           0  
4           0           0  


In [4]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['dayofweek'] = df['date'].dt.dayofweek
df['quarter'] = df['date'].dt.quarter
df['is_working_day'] = ((df['is_holiday'] == 0) & (df['is_weekend'] == 0)).astype(int)

In [6]:
# Train-Test Split
train_data = df[df['year'] <= 2023].copy()
test_data = df[df['year'] == 2024].copy()

In [7]:
print(f"🧪 MODEL TESTING SETUP:")
print(f" Training data: {len(train_data):,} records (2021-2023)")
print(f" Test data: {len(test_data):,} records (2024)")
print(f" Sections: {df['section_id'].nunique()}")

🧪 MODEL TESTING SETUP:
 Training data: 6,570 records (2021-2023)
 Test data: 2,196 records (2024)
 Sections: 6


In [9]:
# Step 1: Analyze training data
# =============================================================
print("\n=== TRAINING DATA ANALYSIS (2021-2023) ===")
working_train = train_data[train_data['is_working_day'] == 1]


print("\n📊 CORRELATION ANALYSIS:")
correlations = {}
for section in working_train['section_id'].unique():
    section_data = working_train[working_train['section_id'] == section]
    if len(section_data) > 20:
        corr = section_data['employees_on_duty'].corr(section_data['total_task_time_minutes'])
        correlations[section] = corr
        print(f" {section}: {correlations[section]:.3f}")


print(f"\n📈 TRAINING STATISTICS:")
stats = working_train.groupby('section_id').agg({
    'employees_on_duty': ['count', 'mean', 'std', 'min', 'max'],
    'total_task_time_minutes': ['mean', 'std']
}).round(2)


for section in stats.index:
    emp_stats = stats.loc[section, 'employees_on_duty']
    task_stats = stats.loc[section, 'total_task_time_minutes']
    print(f" {section}:")
print(f" Working days: {emp_stats['count']:.0f}")
print(f" Employees: {emp_stats['mean']:.1f} ± {emp_stats['std']:.1f} (range: {emp_stats['min']:.0f}-{emp_stats['max']:.0f})")
print(f" Task time: {task_stats['mean']:.0f} ± {task_stats['std']:.0f} minutes")


=== TRAINING DATA ANALYSIS (2021-2023) ===

📊 CORRELATION ANALYSIS:
 SEC-001: 0.892
 SEC-002: 0.908
 SEC-003: 0.873
 SEC-004: 0.902
 SEC-005: 0.983
 SEC-006: 0.969

📈 TRAINING STATISTICS:
 SEC-001:
 SEC-002:
 SEC-003:
 SEC-004:
 SEC-005:
 SEC-006:
 Working days: 727
 Employees: 6.5 ± 2.6 (range: 0-19)
 Task time: 2504 ± 1087 minutes


In [10]:
# =============================================================
# Step 2: Build employee prediction models
# =============================================================
print("\n=== BUILDING MODELS FROM TRAINING DATA ===")


employee_models = {}
for section in working_train['section_id'].unique():
    section_data = working_train[working_train['section_id'] == section]


    if len(section_data) > 30:
        X = section_data[['total_task_time_minutes']].values
        y = section_data['employees_on_duty'].values


        model = HuberRegressor(epsilon=1.35)
        model.fit(X, y)


        valid_data = section_data[section_data['employees_on_duty'] > 0].copy()
        valid_data['task_per_emp'] = valid_data['total_task_time_minutes'] / valid_data['employees_on_duty']


        employee_models[section] = {
            'regression_model': model,
            'avg_task_per_employee': valid_data['task_per_emp'].median(),
            'min_employees': section_data['employees_on_duty'].min(),
            'max_employees': section_data['employees_on_duty'].max(),
            'training_r2': model.score(X, y)
        }


        print(f" {section}: R² = {model.score(X, y):.3f}, Avg efficiency = {valid_data['task_per_emp'].median():.1f} min/emp")




=== BUILDING MODELS FROM TRAINING DATA ===
 SEC-001: R² = 0.795, Avg efficiency = 345.3 min/emp
 SEC-002: R² = 0.824, Avg efficiency = 374.1 min/emp
 SEC-003: R² = 0.762, Avg efficiency = 352.4 min/emp
 SEC-004: R² = 0.814, Avg efficiency = 355.5 min/emp
 SEC-005: R² = 0.967, Avg efficiency = 393.3 min/emp
 SEC-006: R² = 0.935, Avg efficiency = 388.7 min/emp


In [11]:
# Step 3: Forecast 2024 task time with Prophet
# =============================================================
print("\n=== FORECASTING 2024 TASK TIME ===")
task_forecasts = {}
task_time_models = {}


for section in working_train['section_id'].unique():
    print(f" Forecasting {section}...")
    section_data = working_train[working_train['section_id'] == section].copy()


    if len(section_data) < 50:
        print(f" ⚠️ Insufficient training data")
        continue


    prophet_df = section_data[['date', 'total_task_time_minutes']].copy()
    prophet_df.columns = ['ds', 'y']
    prophet_df = prophet_df.sort_values('ds')


    model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=False,
        daily_seasonality=False,
        changepoint_prior_scale=0.05,
        seasonality_prior_scale=8.0
    )


    model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
    model.add_seasonality(name='quarterly', period=91.25, fourier_order=3)
    model.fit(prophet_df)


    test_working = test_data[(test_data['section_id'] == section) & (test_data['is_working_day'] == 1)]['date'].values


    if len(test_working) > 0:
        future_df = pd.DataFrame({'ds': test_working})
        forecast = model.predict(future_df)
        forecast['yhat'] = forecast['yhat'].clip(lower=0)


        task_forecasts[section] = {
        'dates': test_working,
        'predictions': forecast['yhat'].values
        }


        task_time_models[section] = model
        print(f" → Forecasted {len(test_working)} working days")


=== FORECASTING 2024 TASK TIME ===
 Forecasting SEC-001...


13:58:36 - cmdstanpy - INFO - Chain [1] start processing
13:58:38 - cmdstanpy - INFO - Chain [1] done processing
13:58:38 - cmdstanpy - INFO - Chain [1] start processing


 → Forecasted 241 working days
 Forecasting SEC-002...


13:58:38 - cmdstanpy - INFO - Chain [1] done processing
13:58:39 - cmdstanpy - INFO - Chain [1] start processing


 → Forecasted 241 working days
 Forecasting SEC-003...


...

 → Forecasted 241 working days
 Forecasting SEC-005...


13:58:40 - cmdstanpy - INFO - Chain [1] done processing
13:58:40 - cmdstanpy - INFO - Chain [1] start processing


 → Forecasted 241 working days
 Forecasting SEC-006...


13:58:40 - cmdstanpy - INFO - Chain [1] done processing


 → Forecasted 241 working days


In [13]:
# Step 4: Convert task time forecasts into employee predictions
# =============================================================
print("\n=== CONVERTING TO 2024 EMPLOYEE PREDICTIONS ===")


employee_predictions = {}
for section, forecast_data in task_forecasts.items():
    if section not in employee_models:
        continue


    model_info = employee_models[section]
    dates = forecast_data['dates']
    predicted_task_time = forecast_data['predictions']


    predicted_employees_reg = model_info['regression_model'].predict(predicted_task_time.reshape(-1, 1))
    predicted_employees_eff = predicted_task_time / model_info['avg_task_per_employee']


    r2_val = model_info['training_r2']
    weight_reg = 0.8 if r2_val > 0.9 else 0.6


    final_predictions = weight_reg * predicted_employees_reg + (1 - weight_reg) * predicted_employees_eff
    final_predictions = np.round(final_predictions).astype(int)
    final_predictions = np.clip(
        final_predictions,
        max(1, model_info['min_employees']),
        int(model_info['max_employees'] * 1.2)
    )


    employee_predictions[section] = {
        'dates': dates,
        'predictions': final_predictions
    }


=== CONVERTING TO 2024 EMPLOYEE PREDICTIONS ===


In [19]:
# =============================================================
# Step 5: Create predictions DataFrame
# =============================================================
predictions_df = test_data.copy()
predictions_df['predicted_employee_count'] = 0

for section, pred_data in employee_predictions.items():
    for date, pred in zip(pred_data['dates'], pred_data['predictions']):
        mask = (predictions_df['section_id'] == section) & (predictions_df['date'] == date)
        predictions_df.loc[mask, 'predicted_employee_count'] = pred

predictions_df = predictions_df[['date', 'section_id', 'employees_on_duty', 
                                'total_task_time_minutes', 'predicted_employee_count', 'is_working_day']]


In [ ]:

# =============================================================
# Step 6: Evaluate predictions
# =============================================================
print("\n=== COMPREHENSIVE EVALUATION ===")
working_predictions = predictions_df[predictions_df['is_working_day'] == 1].copy()

mae_overall = mean_absolute_error(working_predictions['employees_on_duty'], working_predictions['predicted_employee_count'])
rmse_overall = np.sqrt(mean_squared_error(working_predictions['employees_on_duty'], working_predictions['predicted_employee_count']))
r2_overall = r2_score(working_predictions['employees_on_duty'], working_predictions['predicted_employee_count'])

print(f"🎯 OVERALL PERFORMANCE:")
print(f"   MAE: {mae_overall:.2f} employees")
print(f"   RMSE: {rmse_overall:.2f} employees")
print(f"   R²: {r2_overall:.3f}")
print(f"   Records evaluated: {len(working_predictions):,}")

print(f"\n📊 SECTION-WISE PERFORMANCE:")
section_results = []
for section in working_predictions['section_id'].unique():
    section_data = working_predictions[working_predictions['section_id'] == section]
    if len(section_data) > 0:
        mae_section = mean_absolute_error(section_data['employees_on_duty'], section_data['predicted_employee_count'])
        rmse_section = np.sqrt(mean_squared_error(section_data['employees_on_duty'], section_data['predicted_employee_count']))
        r2_section = r2_score(section_data['employees_on_duty'], section_data['predicted_employee_count'])

        valid_section = section_data[section_data['employees_on_duty'] > 0].copy()
        avg_task_per_emp = (valid_section['total_task_time_minutes'] / valid_section['employees_on_duty']).mean()

        section_results.append({
            'Section': section,
            'MAE': mae_section,
            'RMSE': rmse_section,
            'R²': r2_section,
            'Avg_Task_Per_Employee': avg_task_per_emp,
            'Working_Days': len(section_data),
            'Actual_Avg': section_data['employees_on_duty'].mean(),
            'Predicted_Avg': section_data['predicted_employee_count'].mean()
        })

        print(f"   {section}:")
        print(f"      MAE: {mae_section:.2f}, RMSE: {rmse_section:.2f}, R²: {r2_section:.3f}")
        print(f"      Avg task/employee: {avg_task_per_emp:.1f} minutes")
        print(f"      Actual avg: {section_data['employees_on_duty'].mean():.1f}, Predicted avg: {section_data['predicted_employee_count'].mean():.1f}")

section_results_df = pd.DataFrame(section_results)


In [ ]:

# =============================================================
# Step 7: Visualizations
# =============================================================
print("\n=== CREATING VISUALIZATIONS ===")
fig = plt.figure(figsize=(20, 16))

plt.subplot(3, 3, 1)
plt.scatter(working_predictions['employees_on_duty'], working_predictions['predicted_employee_count'], alpha=0.6, s=20)
plt.plot([0, working_predictions['employees_on_duty'].max()], [0, working_predictions['employees_on_duty'].max()], 'r--', alpha=0.8)
plt.xlabel('Actual Employees')
plt.ylabel('Predicted Employees')
plt.title(f'Overall Prediction Accuracy\nR² = {r2_overall:.3f}')
plt.grid(True, alpha=0.3)

plt.subplot(3, 3, 2)
plt.bar(section_results_df['Section'], section_results_df['MAE'])
plt.xlabel('Section')
plt.ylabel('MAE (employees)')
plt.title('Mean Absolute Error by Section')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

plt.subplot(3, 3, 3)
plt.bar(section_results_df['Section'], section_results_df['R²'])
plt.xlabel('Section')
plt.ylabel('R² Score')
plt.title('R² Score by Section')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

plt.subplot(3, 3, 4)
plt.bar(section_results_df['Section'], section_results_df['Avg_Task_Per_Employee'], color='skyblue')
plt.xlabel('Section')
plt.ylabel('Minutes per Employee')
plt.title('Average Task Time per Employee')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

plt.subplot(3, 3, 5)
x = np.arange(len(section_results_df))
width = 0.35
plt.bar(x - width/2, section_results_df['Actual_Avg'], width, label='Actual', alpha=0.8)
plt.bar(x + width/2, section_results_df['Predicted_Avg'], width, label='Predicted', alpha=0.8)
plt.xlabel('Section')
plt.ylabel('Average Employees')
plt.title('Actual vs Predicted Average Employees')
plt.xticks(x, section_results_df['Section'], rotation=45)
plt.legend()
plt.grid(True, alpha=0.3)

sections = working_predictions['section_id'].unique()[:4]
for i, section in enumerate(sections, 6):
    plt.subplot(3, 3, i)
    section_data = working_predictions[working_predictions['section_id'] == section]
    plt.scatter(section_data['employees_on_duty'], section_data['predicted_employee_count'], alpha=0.6)
    plt.plot([0, section_data['employees_on_duty'].max()], [0, section_data['employees_on_duty'].max()], 'r--', alpha=0.8)
    plt.xlabel('Actual')
    plt.ylabel('Predicted')
    plt.title(f'{section} Predictions')
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('model_validation_results.png', dpi=300, bbox_inches='tight')
plt.show()

plt.figure(figsize=(15, 10))
working_predictions['year_month'] = working_predictions['date'].dt.to_period('M')
monthly_comparison = working_predictions.groupby(['year_month', 'section_id']).agg({
    'employees_on_duty': 'mean',
    'predicted_employee_count': 'mean'
}).reset_index()

sections = monthly_comparison['section_id'].unique()
for i, section in enumerate(sections):
    plt.subplot(2, 3, i+1)
    section_monthly = monthly_comparison[monthly_comparison['section_id'] == section]
    plt.plot(section_monthly['year_month'].astype(str), section_monthly['employees_on_duty'], 'o-', label='Actual', linewidth=2)
    plt.plot(section_monthly['year_month'].astype(str), section_monthly['predicted_employee_count'], 's-', label='Predicted', linewidth=2)
    plt.title(f'{section} - Monthly Average')
    plt.xlabel('Month (2024)')
    plt.ylabel('Avg Employees')
    plt.legend()
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('monthly_trends_comparison.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:

# =============================================================
# Step 8: Save results
# =============================================================
predictions_df.to_csv('2024_validation_predictions.csv', index=False)
section_results_df.to_csv('validation_performance_by_section.csv', index=False)

print("\n✅ VALIDATION COMPLETE!")
print(f"📁 Files saved:")
print(f"   - 2024_validation_predictions.csv")
print(f"   - validation_performance_by_section.csv")
print(f"   - model_validation_results.png")
print(f"   - monthly_trends_comparison.png")

print("\n📊 FINAL VALIDATION SUMMARY:")
print(section_results_df.round(3))

if mae_overall < 1.0:
    print("\n✅ Model performance acceptable! Ready for 2025 forecasting.")
else:
    print("\n⚠️  Consider model improvements before 2025 forecasting.")
